In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

***Question 1***

La charge de la batterie est très importante afin de connaître l'autonomie du système. On a 
$$
I = \dfrac{\partial Q}{\partial t}
$$
En discrétisé on a, à l'instant i 
$$
I_i = \frac{Q_i}{\Delta t}
$$
Ainsi on trouve bien que pour tous les instants entérieurs, on doit obtenir que 
$$
\sum_{i=0}^{N}\Delta t I_i \leq Q_bat
$$

***Question 2***

$a_1$ : paramétre de frottement statique. Ce paramètre varie en fonction du poids du cycliste

$a_2$ : terme de frottement de l'air ( hypothèse d'écoulement laminaire). Ce paramètre varie en fonction de la surface du cycliste

$a_3$ : terme de couple moteur. Ne varie pas en fonction du cycliste

$a_4$ : terme de couple humain ( couple du cycliste sur les pédales ). Varie en fonction de la force musculaire du cycliste

$-g\gamma (x_i)$ : terme de résistance de la pesenteur au mouvement. Ne dépend pas du cycliste car la masse a été éliminé pour obtenir ces équations

***Question 3***

On veut une assistance optimale, c'est à dire qui permettent d'aller le plus loin possible sans avoir à s'épuiser. Il faut donc chercher à minimiser $\sum_{i=0}^{N}T_i ^2$ ( le couple musculaire ), en farantissant la condition de batterie $\sum_{i=0}^{N}\Delta t I_i \leq Q_{bat}$.

Le problème d'optimisation se réécrit donc :
$$
\underset{T_i,I_i,x_i,v_i,t_i}{min} \sum_{i=0}^{N}T_i ^2  \
$$
Tel que 
$$
v_{i+1} - v_i =  \Delta t(a_1 - a_2vi + a_3Ii + a_4Ti + g\gamma (xi))\\
\sum_{i=0}^{N}\Delta t I_i - Q_{bat} = 0 \\
I_i - I_M \leq 0
$$

***Question 4***

Etude de la convexité du problème : \
On pose
$$
f : (T_i,I_i,x_i,v_i,t_i) \rightarrow \underset{T_i,I_i,x_i,v_i,t_i}{min} \sum_{i=0}^{N}T_i ^2
$$
Si l'on considère le couple de pédalage constant, on a :
$$
\nabla ^2 f(T_i,I_i,x_i,v_i,t_i) = 0 
$$
Donc 
$$
\nabla ^2 f(T_i,I_i,x_i,v_i,t_i) \ge 0 
$$
Ainsi f est bien convexe.

Bon c'est clairement faux mais je ne savais pas quoi mettre, sinon je dis juste que ça depend de la tête de Ti

***Identification du modèle dynamique***

***Question 5***

Formulation du problème des moindres carrés: \
On pose,

$$
A = 
\begin{pmatrix}
1 & -v_0 & I_0 & T_0 \\
1 & -v_1 & I_1 & T_1 \\
... & ... & ... & ... \\
1 & -v_n & I_n & T_n \\
\end{pmatrix} 
$$
\
$$
B = 
\begin{pmatrix}
v_1 - v_0 \\ v_2 - v_1 \\ ... \\ v_{n+1} - v{n}
\end{pmatrix}
$$
\
$$
X = \begin{pmatrix}
a_1 \\ a_2 \\ a_3 \\ a_4
\end{pmatrix}
$$

Le problème des moindres carrés s'écrit alors :

$$
AX = B
$$

***Question 6***

In [101]:
with open('data_velo.csv', encoding = "utf8") as f :
    df = pd.read_csv(f)
    #print(df.head())
    p = len(df.columns) - 1
    n = len(df.axes[0])
    A = np.zeros((n,p))
    B = np.zeros((n,1))
    V = df['Vitesse [m/s]']
    I = df['I [A]']
    T = df['Couple pédale [Nm]']
    dt = 0.01
    #print(V)
    for i in range(len(V) - 1):
        B[i] = V[i+1] - V[i]
    for j in range(n):
        A[j][0] = 1
        A[j][1] = -V[j]
        A[j][2] = I[j]
        A[j][3] = T[j]
    A = A * dt

    X = np.linalg.lstsq(A, B, rcond = -1)[0]
    print(f"Les solutions obtenues par la méthodes des moindres carrés sont a1 = {X[0]}, a2 = {X[1]}, a3 = {X[2]}, a4 = {X[3]}")

Les solutions obtenues par la méthodes des moindres carrés sont a1 = [-0.29498251], a2 = [0.021807], a3 = [0.02869084], a4 = [0.0143385]


***Question 7***

***a)***

On pose :
$$
C = A_k^TA_k \\
U = a_{k+1} \\
V = a_{k+1}^T \\
\bar{C} = C + UV^T
$$
On suppose $\bar{C}$ inversible, on peut donc appliquer la formule de Sherman-Morrison-Woodbury :
$$
\bar{C}^{-1} = C^{-1} - C^{-1}U(I + V^TA^{-1}U)^{-1}V^TA^{-1}
$$
on trouve finalement bien que :
$$
    (A_k^T A_k + a_{k+1}a_{k+1}^T)^{−1} = (A_k^T A_k)^{−1} − K_k a_{k+1}^T(A_k^T A_k)^{−1}
$$
Avec :
$$
Kk = γk(A_k^T A_k)^{−1}a_{k+1} 
où 
γk = 1/(1 + a_{k+1}^T(A_k^T A_k)^{−1}a_{k+1}) 
$$
Or par multiplication de matrice :
$$
A_{k+1}A_{k+1}^T = (A_k^T, ak+1)^T(A_kT, ak+1) = A_kA_k^T + a_{k+1}^Ta_{k+1}
$$
On a finalement :
$$
(A_{k+1}^TA_{k+1})^{−1} = (A_k^T A_k + a_{k+1}a_{k+1}^T)^{−1}
$$
D'où :
$$
\boxed{
(A_{k+1}^TA_{k+1})^{−1} = (A_k^T A_k + a_{k+1}a_{k+1}^T)^{−1} = (A_k^T A_k)^{−1} − K_k a_{k+1}^T(A_k^T A_k)^{−1}}
$$


***b)***

On sait que : 
$$
x_k = (A_{k}^TA_{k})^{−1}A_{k}^{T}b_{k}
$$
Alors on déduit:
$$
\begin{align*}
x_{k+1} &= (A_{k+1}^TA_{k+1})^{−1}A_{k+1}^{T}b_{k+1} \\
&= [(A_{k}^TA_{k})^{−1} − K_k a_{k+1}^T(A_k^T A_k)^{−1}][A_{k}^{T}b_{k} + a_{k+1} \tilde{b}] \\ 
&= x_k + K_k \tilde{b} (1 + a_{k+1}^T(A_k^T A_k)^{−1}a_{k+1}) - K_k a_{k+1}^{T} [(A_k^{T}A_k)^{-1}A_k^T b_k]-  K_k \tilde{b} a_{k+1}^{T}(A_k^T A_k)^{-1} a_{k+1} \\ 
&= x_k + K_k (\tilde{b} - a_{K+1}^{T} x_k)
\end{align*}
$$

On a alors identifié $x_k = (A_k^{T}A_k)^{-1}A_k^T b_k$ dans l'avant-dernier terme, le dernier se simplifiant avec un des deux premiers.

D'où notre résultat:
$$
\boxed{x_{k+1} = x_k + K_k (\tilde{b} - a_{K+1}^{T} x_k)}
$$

***c)***

Un algorithme permettant de calculer itérativement xk+1, et qui ne nécessite pas d’inversion de matrice au cours des itérations est l'algorithme BFGS.

In [102]:
#Paramétrage initial
A0 = A[:4, :5]
Uk = inv(np.dot(A0, np.transpose(A0)))
xk = np.array([0., 0., 0., 0.])
N = 3251

***Question 8***

In [103]:
#On itère N fois
for k in range(N):
    ak = A[k, :]
    bk = B[k]
    Kk = np.dot(Uk, ak) / (1 + np.dot(np.dot(np.transpose(ak),Uk),ak))
    Uk = Uk - np.dot(np.dot(Kk,np.transpose(ak)),Uk)
    xk = xk + Kk*(bk - np.dot(np.transpose(ak),xk))

#Affichage du résultat final:
print(xk)

[ 0.00972769 -0.02073249 -0.0021763   0.01320493]


On trouve au final des résultats différents avec la question 6, mais on peut déjà avoir "en direct" des résultats plutôt fiables.
